In [1]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import ast
import time

# -------------------------------------------------------------
# Paths
# -------------------------------------------------------------
coord_path = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv"
out_dir = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
os.makedirs(out_dir, exist_ok=True)

# -------------------------------------------------------------
# Load coordinates
# -------------------------------------------------------------
df = pd.read_csv(coord_path)

# Sample size option (set to None to process all coordinates)
sample_size = None  # Change to None for full dataset

def parse_coord(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return None

df["COORDINATES"] = df["COORDINATES"].apply(parse_coord)
df = df[df["COORDINATES"].notna()].copy()

# Apply sample size if specified
if sample_size is not None:
    df = df.head(sample_size)
    print(f"Processing sample of {len(df)} coordinates (sample_size={sample_size})")
else:
    print(f"Processing all {len(df)} coordinates")

# -------------------------------------------------------------
# Quiet GET wrapper
# -------------------------------------------------------------
def safe_get(url, params=None):
    try:
        r = requests.get(url, params=params, timeout=12)
        if r.status_code == 200:
            return r.json()
        return None
    except:
        return None

# -------------------------------------------------------------
# FEMA NFHL Fetcher
# -------------------------------------------------------------
def fetch_nfhl(lat, lon):
    # Use the ArcGIS REST service for flood hazard zones (layer 28)
    url = "https://hazards.fema.gov/arcgis/rest/services/public/NFHL/MapServer/28/query"
    
    params = {
        "f": "json",
        "where": "1=1",
        "geometry": f"{lon},{lat}",
        "geometryType": "esriGeometryPoint",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "FLD_ZONE,SFHA_TF,STATIC_BFE,V_DATUM,ZONE_SUBTY,SOURCE_CIT",
        "returnGeometry": "false"
    }

    js = safe_get(url, params)
    if not js or "features" not in js or not js["features"]:
        # No flood zone at this location → default nulls
        return None

    feature = js["features"][0]
    attrs = feature.get("attributes", {})

    return {
        "NFHL_FLD_ZONE": attrs.get("FLD_ZONE"),
        "NFHL_SFHA": attrs.get("SFHA_TF"),
        "NFHL_STATIC_BFE": attrs.get("STATIC_BFE"),
        "NFHL_V_DATUM": attrs.get("V_DATUM"),
        "NFHL_ZONE_SUBTYPE": attrs.get("ZONE_SUBTY"),
        "NFHL_SOURCE_CIT": attrs.get("SOURCE_CIT")
    }

# -------------------------------------------------------------
# Loop all structures
# -------------------------------------------------------------
rows = []
print("\nStarting FEMA NFHL enrichment...\n")

for _, r in tqdm(df.iterrows(), total=len(df), desc="FEMA NFHL"):
    sid = r["STRUCTURE_ID"]
    lat, lon = r["COORDINATES"]

    entry = {
        "STRUCTURE_ID": sid,
        "NFHL_FLD_ZONE": None,
        "NFHL_SFHA": None,
        "NFHL_STATIC_BFE": None,
        "NFHL_V_DATUM": None,
        "NFHL_ZONE_SUBTYPE": None,
        "NFHL_SOURCE_CIT": None
    }

    result = fetch_nfhl(lat, lon)
    if result:
        entry.update(result)

    rows.append(entry)
    time.sleep(0.15)   # FEMA REST API → slight delay recommended

# -------------------------------------------------------------
# Save CSV
# -------------------------------------------------------------
df_out = pd.DataFrame(rows)
out_path = os.path.join(out_dir, "fema_nfhl_enrichment.csv")
df_out.to_csv(out_path, index=False)

print(f"\nSaved FEMA NFHL flood hazard enrichment for {len(df_out)} structures:")
print(out_path)

Processing all 4914 coordinates

Starting FEMA NFHL enrichment...



FEMA NFHL: 100%|██████████| 4914/4914 [1:09:27<00:00,  1.18it/s]


Saved FEMA NFHL flood hazard enrichment for 4914 structures:
C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\fema_nfhl_enrichment.csv
